# IN-STK5000/9000 - Medical Project

## Part 1 - Historical Data

Discovering structure in the data. It is uncertain if the symptoms present are all due to
the same disease, or if they are different conditions with similar symptoms. (a) looking at the
data (including symptoms), estimate whether a single-cause model is more likely than a multiplecause model. You can use anything, ranging from histograms or simple clustering algorithms
to a hierarchical Bayesian model. 

For the clustering part:
* https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#sphx-glr-auto-examples-cluster-plot-kmeans-silhouette-analysis-py
* https://scikit-learn.org/stable/modules/biclustering.html

(b) Try and determine whether some particular factors are
important for disease epidemiology and may require further investigations.
You need to be able to validate your findings either through a holdout-set methodology,
appropriately used statistical tests, or Bayesian model comparison.

* Shap values  

* Meaning of epidemiology? Do we predict disease or symptoms? Using which variables?

Measuring the effect of actions. We also observe the effects of two different therapeutic
interventions, one of which is placebo, and the other is an experimental drug. Try and measure
the effectiveness of the placebo versus the active treatment. Are there perhaps cases where the
active treatment is never effective, or should it always be recommended?
1

* Synthetic control 

## Part 2 - Improved Policy

## Part 3 - Adaptive experiment design